In [ ]:
import gc
import json
import math
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
#Copied from previous comp 1st place model: https://www.kaggle.com/code/hoyso48/1st-place-solution-training
class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs, mask=None):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:,None,:]
        return inputs * nn

class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self, 
        kernel_size=17,
        dilation_rate=1,
        use_bias=False,
        depthwise_initializer='glorot_uniform',
        name='', **kwargs):
        super().__init__(name=name,**kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
                            kernel_size,
                            strides=1,
                            dilation_rate=dilation_rate,
                            padding='valid',
                            use_bias=use_bias,
                            depthwise_initializer=depthwise_initializer,
                            name=name + '_dwconv')
        self.supports_masking = True
        
    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x

def Conv1DBlock(channel_size,
          kernel_size,
          dilation_rate=1,
          drop_rate=0.0,
          expand_ratio=2,
          se_ratio=0.25,
          activation='swish',
          name=None):
    '''
    efficient conv1d block, @hoyso48
    '''
    if name is None:
        name = str(tf.keras.backend.get_uid("mbblock"))
    # Expansion phase
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs

        x = tf.keras.layers.Dense(
            channels_expand,
            use_bias=True,
            activation=activation,
            name=name + '_expand_conv')(inputs)

        # Depthwise Convolution
        x = CausalDWConv1D(kernel_size,
            dilation_rate=dilation_rate,
            use_bias=False,
            name=name + '_dwconv')(x)

        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

        x  = ECA()(x)

        x = tf.keras.layers.Dense(
            channel_size,
            use_bias=True,
            name=name + '_project_conv')(x)

        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1), name=name + '_drop')(x)

        if (channels_in == channel_size):
            x = tf.keras.layers.add([x, skip], name=name + '_add')
        return x

    return apply

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
def TCNBlock(filters, kernel_size, dilation_rate, drop_rate=0.0, activation='relu', name=None):
    """
    Temporal Convolutional Block with Bahdanau Attention.
    """
    if name is None:
        name = str(tf.keras.backend.get_uid("tcnblock"))
    
    attention_layer = BahdanauAttention(filters)
    
    def apply(inputs):
        skip = inputs

        # 1. Causal Convolution
        x = CausalDWConv1D(kernel_size, dilation_rate=dilation_rate, name=name + '_dwconv')(inputs)
        
        # 2. Batch Normalization and Activation
        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn1')(x)
        x = tf.keras.layers.Activation(activation, name=name + '_act1')(x)

        # 3. Efficient Channel Attention
        x = ECA()(x)

        # 4. Apply Bahdanau Attention
        query = tf.keras.layers.Dense(filters)(x)
        x, attention_weights = attention_layer(query, x)

        # 5. Pointwise Convolution
        x = tf.keras.layers.Conv1D(filters, 1, activation=activation, name=name + '_conv1')(x)
        
        # 6. Another round of Batch Normalization
        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn2')(x)
        
        # 7. Dropout for regularization
        x = tf.keras.layers.Dropout(drop_rate, name=name + '_drop')(x)
        
        # 8. Project input to the right shape for residual connection
        if tf.keras.backend.int_shape(skip)[-1] != filters:
            skip = tf.keras.layers.Conv1D(filters, 1, name=name + '_residual_proj')(skip)
            
        # 9. Residual connection
        x = tf.keras.layers.add([x, skip], name=name + '_add')
        
        return x

    return apply
